In [ ]:
from fbprophet import Prophet
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries

In [2]:
def save_dataset(symbol):
    api_key = 'P33J9T7IVI663Y0A'

    ts = TimeSeries(key=api_key, output_format='pandas')
    data, meta_data = ts.get_daily(symbol, outputsize='full')

    data.to_csv(f'./DataOutput/{symbol}_daily.csv')
    print (f'./DataOutput/{symbol}_daily.csv')

In [3]:
def csv_to_dataset_openclose(csv_path):
    data = pd.read_csv(csv_path)
    dateList = []
    yList = []
    row_count = len(data['1. open'])

    for i in range (row_count):
      dateList.append (str(data.date[i]) + ' 09:30:00')
      dateList.append (str(data.date[i]) + ' 16:00:00')
      yList.append (data['1. open'][i])
      yList.append (data['4. close'][i])

      data_openclose = pd.DataFrame ({'ds':dateList,'y':yList})

    return data_openclose

In [4]:
save_dataset('AMZN')

./DataOutput/AMZN_daily.csv


In [5]:
data = csv_to_dataset_openclose('./DataOutput/AMZN_daily.csv')

In [ ]:
data.head()

In [ ]:
model = Prophet(daily_seasonality=True)
model.fit(data)
data.tail()

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


KeyError: 'metric_file'

Exception ignored in: 'stanfit4anon_model_f5236004a3fd5b8429270d00efcc0cf9_9045703871880297330._set_stanargs_from_dict'
Traceback (most recent call last):
  File "C:\Users\Ahmad Abo Alafa\anaconda3\lib\site-packages\pystan\model.py", line 550, in optimizing
    ret, sample = fit._call_sampler(stan_args)
KeyError: 'metric_file'


In [ ]:
future_df = model.make_future_dataframe(periods=365)
future_df

In [ ]:
predictions = model.predict(future_df)
predictions

In [ ]:
model.plot(predictions)
model.plot_components(predictions)